<a href="https://colab.research.google.com/github/Welsee/tinyML/blob/master/colab_camare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opencv-python-headless
!pip install torch
!pip install git https://github.com/ultralytics/yolov5.git


     | 369.4 kB 1.5 MB/s 0:00:00
  ERROR: Cannot unpack file /tmp/pip-unpack-gw8gnscp/yolov5.git (downloaded from /tmp/pip-req-build-cxkzf5_w, content-type: text/html; charset=utf-8); cannot detect archive format
ERROR: Cannot determine archive format of /tmp/pip-req-build-cxkzf5_w


In [ ]:
import cv2
import torch
from IPython.display import display, Image, clear_output
from yolov5.models.experimental import attempt_load
from yolov5.utils.general import non_max_suppression, scale_coords
from yolov5.utils.plots import plot_one_box

# YOLOv5 모델 로드
model = attempt_load('yolov5s.pt', map_location='cuda')  # 미리 학습된 YOLOv5 모델을 사용하거나 직접 학습된 모델을 지정합니다.

# 웹캠 열기
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 이미지를 YOLOv5 모델에 입력으로 제공
    img = torch.from_numpy(frame).cuda().float()  # 이미지를 PyTorch Tensor로 변환
    img /= 255.0  # 이미지 정규화 (0-1 사이 값으로 변환)
    if img.ndimension() == 3:
        img = img.unsqueeze(0)

    # 객체 검출 수행
    prediction = model(img)[0]

    # 검출된 객체에 대한 후처리 (비최대 억제 등)
    prediction = non_max_suppression(prediction, conf_thres=0.5, iou_thres=0.5)

    # 검출된 객체를 원본 이미지에 그리기
    for det in prediction[0]:
        if det is not None and len(det):
            det[:, :4] = scale_coords(img.shape[2:], det[:, :4], frame.shape).round()
            for *xyxy, conf, cls in det:
                label = f'{model.names[int(cls)]} {conf:.2f}'
                plot_one_box(xyxy, frame, label=label, color=(0, 255, 0))

    # 웹캠 영상 표시
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # OpenCV 이미지를 RGB로 변환
    display(Image(data=cv2.imencode('.jpg', frame)[1]))
    clear_output(wait=True)

cap.release()
cv2.destroyAllWindows()





ModuleNotFoundError: ignored